In [1]:
from downloading_BiGG_models import *
from directory_infomation import *
from functions_and_dicts_data_preprocessing_GNN import *
import os

### For all models, map the AA sequences to the df_KM DataFrames for all Uniprot IDs and store all sequences with Uniprot IDs in a DataFrame. Then I can calculate the Unirep vectors for all sequences at once:

In [2]:
model_IDs = os.listdir(join(datasets_dir, "BiGG_GSM"))
len(model_IDs)

47

In [3]:
def map_sequences_to_df_Km(df_KM, df_sequences, model_ID):
    Uniprot_df = pd.read_csv(join(datasets_dir, "BiGG_GSM", model_ID,
                                  "Uniprot_Mapping_" + model_ID + ".csv"), sep = ";")
    
    df_KM["Sequence"] = np.nan

    for ind in df_KM.index:
        UID = df_KM["Uniprot ID"][ind]
        if not pd.isnull(UID):
            seq = list(Uniprot_df["Sequence"].loc[Uniprot_df["Uniprot ID"] == UID])[0]
            df_KM["Sequence"][ind] = seq
            df_sequences = df_sequences.append({"Uniprot ID": UID, "Sequence" : seq}, ignore_index=True)
    
    df_KM.to_pickle(join(datasets_dir, "BiGG_GSM", model_ID, "df_KM_checkpoint_" + model_ID + ".pkl"))
    return(df_sequences)

In [4]:
df_sequences = pd.DataFrame(columns = ["Uniprot ID", "Sequence"])

for model_ID in model_IDs:
    df_KM = pd.read_pickle(join(datasets_dir, "BiGG_GSM", model_ID, "df_KM_checkpoint_" + model_ID + ".pkl"))
    df_sequences = map_sequences_to_df_Km(df_KM, df_sequences, model_ID)

df_sequences.to_pickle(join(datasets_dir, "BiGG_data", "df_sequences.pkl"))
df_sequences.drop_duplicates()

,Uniprot ID,Sequence
0,P0A796,MIKKIGVLTSGGDAPGMNAAIRGVVRSALTEGLEVMGIYDGYLGLY...
2,P06999,MVRIYTLTLAPSLDSATITPQIYPEGKLRCTAPVFEPGGGGINVAR...
4,P0A6T1,MKNINPTQTAAWQALQKHFDEMKDVTIADLFAKDGDRFSKFSATFD...
6,P0A799,MSVIKMTDLDLAGKRVFIRADLNVPVKDGKVTSDARIRASLPTIEL...
10,P52697,MKQTVYIASPESQQIHVWNLNHEGALTLTQVVDVPGQVQPMVVSPD...
...,...,...
178747,Q04119,MVVVGKSEVRNVSMSRPKKKSLIAILSTCVLFFLVFIIGAKFQYVS...
178755,P39979,MSNEEPEKMVNDRIVVKAIEPKDEEAWNKLWKEYQGFQKTVMPPEV...
178775,P38295,MSEVSKWPAINPFHWGYNGTVSHIVGENGSIKLHLKDNKEQVDFDE...
178779,Q02891,MFRSGYYPTVTPSHWGYNGTVKHVLGEKGTKSLAFRDSKRQIPLHE...


Concatenate all df_KM DataFrames:

## Creat input files for all substrates

In [5]:
df_KM_all_models = pd.DataFrame(columns = ['BiGG reaction ID', 'reaction name', 'gene_reaction_rule',
                                            'substrate', 'Uniprot ID', 'KEGG ID', 'bigg.metabolite', 'metanetx ID',
                                            'substrate name', 'SMILES', 'Sequence', 'model_ID'])

for model_ID in model_IDs:
    df_KM = pd.read_pickle(join(datasets_dir, "BiGG_GSM", model_ID, "df_KM_checkpoint_" + model_ID + ".pkl"))
    df_KM["model_ID"] = model_ID
    df_KM_all_models = pd.concat([df_KM_all_models, df_KM], ignore_index= True)
df_KM_all_models

,BiGG reaction ID,reaction name,gene_reaction_rule,substrate,Uniprot ID,KEGG ID,bigg.metabolite,metanetx ID,substrate name,SMILES,Sequence,model_ID
0,PFK,Phosphofructokinase,b3916,atp_c,P0A796,C00002,atp,MNXM3,ATP C10H12N5O13P3,NaN,MIKKIGVLTSGGDAPGMNAAIRGVVRSALTEGLEVMGIYDGYLGLY...,e_coli_core
1,PFK,Phosphofructokinase,b3916,f6p_c,P0A796,C05345,f6p,MNXM89621,D-Fructose 6-phosphate,NaN,MIKKIGVLTSGGDAPGMNAAIRGVVRSALTEGLEVMGIYDGYLGLY...,e_coli_core
2,PFK,Phosphofructokinase,b1723,atp_c,P06999,C00002,atp,MNXM3,ATP C10H12N5O13P3,NaN,MVRIYTLTLAPSLDSATITPQIYPEGKLRCTAPVFEPGGGGINVAR...,e_coli_core
3,PFK,Phosphofructokinase,b1723,f6p_c,P06999,C05345,f6p,MNXM89621,D-Fructose 6-phosphate,NaN,MVRIYTLTLAPSLDSATITPQIYPEGKLRCTAPVFEPGGGGINVAR...,e_coli_core
4,PFL,Pyruvate formate lyase,((b0902 and b0903) and b2579),coa_c,NaN,C00010,coa,MNXM12,Coenzyme A,NaN,NaN,e_coli_core
...,...,...,...,...,...,...,...,...,...,...,...,...
216368,r_4693,alcohol acetyltransferase (propanol),YGR177C,s_4233[c],P53296,C05979,NaN,NaN,propanol [cytoplasm],NaN,MEDIEGYEPHITQELIDRGHARRMGHLENYFAVLSRQKMYSNFTVY...,yeast8
216369,r_4693,alcohol acetyltransferase (propanol),YOR377W,s_0373[c],P40353,C00024,a,M,acetyl-CoA [cytoplasm],NaN,MNEIDEKNQAPVQQECLKEMIQNGHARRMGSVEDLYVALNRQNLYR...,yeast8
216370,r_4693,alcohol acetyltransferase (propanol),YOR377W,s_4233[c],P40353,C05979,NaN,NaN,propanol [cytoplasm],NaN,MNEIDEKNQAPVQQECLKEMIQNGHARRMGSVEDLYVALNRQNLYR...,yeast8
216371,r_4693,alcohol acetyltransferase (propanol),YGR015C,s_0373[c],P53208,C00024,a,M,acetyl-CoA [cytoplasm],NaN,MSRLAHNKALPYKIIVDLSFHRTRLPSDVSSLIKFEQRPAIINIHG...,yeast8


In [6]:
all_metnetx_IDs = list(set(df_KM_all_models["metanetx ID"].loc[~pd.isnull(df_KM_all_models["metanetx ID"])]))
all_Uniprot_IDs = list(set(df_KM_all_models["Uniprot ID"].loc[~pd.isnull(df_KM_all_models["Uniprot ID"])]))
len(all_metnetx_IDs), len(all_Uniprot_IDs)

(3344, 24415)

In [7]:
df_subs = pd.DataFrame({"metanetx ID" : df_KM_all_models["metanetx ID"], "KEGG ID" : df_KM_all_models["KEGG ID"],
                       "SMILES" : df_KM_all_models["SMILES"]})
df_subs =df_subs.drop_duplicates().reset_index(drop = True)
df_subs.to_pickle(join(datasets_dir, "BiGG_data", "df_subs.pkl"))
df_subs

,metanetx ID,KEGG ID,SMILES
0,MNXM3,C00002,NaN
1,MNXM89621,C05345,NaN
2,MNXM12,C00010,NaN
3,MNXM23,C00022,NaN
4,MNXM160,C00092,NaN
...,...,...,...
4451,NaN,C00479,NaN
4452,NaN,C00163,NaN
4453,NaN,C00100,NaN
4454,NaN,C06044,NaN


In [11]:
def calculate_and_save_input_matrixes(ind, molecule_ID, save_folder, metanetX_ID):
    extras = np.array([0,0])
          
    [XE, X, A] = create_input_data_for_GNN_for_substrates(substrate_ID = molecule_ID, N = 250,  print_error=True)
    if not A is None:
        np.save(join(save_folder, metanetX_ID + '_X.npy'), X) #feature matrix of atoms/nodes
        np.save(join(save_folder, metanetX_ID + '_XE.npy'), XE) #feature matrix of atoms/nodes and bonds/edges
        np.save(join(save_folder, metanetX_ID + '_A.npy'), A) 
        np.save(join(save_folder, metanetX_ID + '_extras.npy'), extras)
        return(True)
    
    return(False)

def create_input_data_for_GNN_for_substrates(substrate_ID, N, print_error = False):
    try:
        x = create_atom_feature_matrix(mol_name = substrate_ID, N =N)
        if not x is None: 
            a,e = create_bond_feature_matrix(mol_name = substrate_ID, N =N)
            a = np.reshape(a, (N,N,1))
            xe = concatenate_X_and_E(x, e, N = N)
            return([np.array(xe), np.array(x), np.array(a)])
        else:
            if print_error:
                print("Could not create input for substrate ID %s" %substrate_ID)  
            return(None, None, None)
    except:
        return(None, None, None)
    
def calculate_atom_and_bond_feature_vectors_for_SMILES(SMILES, metanetX_ID):
        mol = Chem.MolFromSmiles(SMILES)
        succesfull = False
        if not mol is None:
            calculate_atom_feature_vector_for_mol_file(mol, metanetX_ID)
            succesfull = calculate_bond_feature_vector_for_mol_file(mol, metanetX_ID)
        if succesfull:
            return(True)
        else:
            return(False)
        
            
def calculate_bond_feature_vector_for_mol_file(mol, metanetX_ID):
    N = mol.GetNumBonds()
    bond_list = []
    for i in range(N):
        features = []
        bond = mol.GetBondWithIdx(i)
        features.append(bond.GetBeginAtomIdx()), features.append(bond.GetEndAtomIdx()),
        features.append(str(bond.GetBondType())), features.append(bond.GetIsAromatic()),
        features.append(bond.IsInRing()), features.append(str(bond.GetStereo()))
        bond_list.append(features)
    with open(join(datasets_dir, "mol_feature_vectors", metanetX_ID + "-bonds.txt"), "wb") as fp:   #Pickling
        pickle.dump(bond_list, fp)
        return(True)
    
    return(False)
        
def calculate_atom_feature_vector_for_mol_file(mol, metanetX_ID):
    #get number of atoms N
    N = mol.GetNumAtoms()
    atom_list = []
    for i in range(N):
        features = []
        atom = mol.GetAtomWithIdx(i)
        features.append(atom.GetAtomicNum()), features.append(atom.GetDegree()), features.append(atom.GetFormalCharge())
        features.append(str(atom.GetHybridization())), features.append(atom.GetIsAromatic()), features.append(atom.GetMass())
        features.append(atom.GetTotalNumHs()), features.append(str(atom.GetChiralTag()))
        atom_list.append(features)
    with open(join(datasets_dir, "mol_feature_vectors", metanetX_ID + "-atoms.txt"), "wb") as fp:   #Pickling
        pickle.dump(atom_list, fp)
        
        
def download_SMILES_via_metanetx_ID(metanetx):
    reg_url = "https://www.metanetx.org/chem_info/" + metanetx
    req = Request(url=reg_url, headers=headers) 
    html = str(urlopen(req).read())

    #get SMILES:
    start_smiles = '<tr><td class="smiles">SMILES</td><td class="smiles">'
    if html.find(start_smiles) != -1:
        smiles = html[html.find(start_smiles) + len(start_smiles): ]
        smiles = smiles[ : smiles.find("</td></tr>\\n")]
        if smiles != "&nbsp;":
            return(smiles)
        else:
            return(np.nan)
        
def calculate_substrate_inputs_for_GNN(df_subs):
    df_subs["substrate_available"] = np.nan
    
    for ind in df_subs.index:
        subs_avail = False
        metanetX_ID = df_subs["metanetx ID"][ind]

        if not pd.isnull(metanetX_ID):
            if not pd.isnull(df_subs["KEGG ID"][ind]):
                kegg_id = df_subs["KEGG ID"][ind]
                subs_avail = calculate_and_save_input_matrixes(ind = ind, molecule_ID = kegg_id, 
                                                        save_folder = join(datasets_dir, "BiGG_data", "input_data"),
                                                              metanetX_ID = metanetX_ID)
                if not subs_avail: #try to download SMILES:
                    print("We could not calculate the input for KEGG ID: %s (ind = %s)" % (kegg_id, ind))
                    if not pd.isnull(df_subs["metanetx ID"][ind]):
                        df_subs["SMILES"][ind] = download_SMILES_via_metanetx_ID(metanetx = metanetX_ID)

            if not subs_avail and not pd.isnull(df_subs["SMILES"][ind]):
                SMILES_calculated = calculate_atom_and_bond_feature_vectors_for_SMILES(SMILES = df_subs["SMILES"][ind],
                                                                                metanetX_ID = metanetX_ID)
                if SMILES_calculated:
                    subs_avail = calculate_and_save_input_matrixes(ind = ind, molecule_ID = df_subs["metanetx ID"][ind], 
                                                                save_folder =  join(datasets_dir, "BiGG_data", "input_data"),
                                                                metanetX_ID = metanetX_ID)

                if not subs_avail: #try to download SMILES:
                    print("We could not calculate the SMILES input for index: %s" % ind)

        df_subs["substrate_available"][ind] = subs_avail
        
    return(df_subs)

In [12]:
df_subs = calculate_substrate_inputs_for_GNN(df_subs)

Could not create input for substrate ID C01635
We could not calculate the input for KEGG ID: C01635 (ind = 198)
Could not create input for substrate ID C01638
We could not calculate the input for KEGG ID: C01638 (ind = 250)
Could not create input for substrate ID C01636
We could not calculate the input for KEGG ID: C01636 (ind = 256)
Could not create input for substrate ID C01637
We could not calculate the input for KEGG ID: C01637 (ind = 267)
Could not create input for substrate ID C01639
We could not calculate the input for KEGG ID: C01639 (ind = 335)
Could not create input for substrate ID MNXM47647
We could not calculate the SMILES input for index: 438
Could not create input for substrate ID C02869
We could not calculate the input for KEGG ID: C02869 (ind = 502)
Could not create input for substrate ID C01640
We could not calculate the input for KEGG ID: C01640 (ind = 532)
Could not create input for substrate ID C01641
We could not calculate the input for KEGG ID: C01641 (ind = 550)

Could not create input for substrate ID C01651
We could not calculate the input for KEGG ID: C01651 (ind = 4145)
Could not create input for substrate ID C01652
We could not calculate the input for KEGG ID: C01652 (ind = 4148)
Could not create input for substrate ID C00787
We could not calculate the input for KEGG ID: C00787 (ind = 4150)
Could not create input for substrate ID C01653
We could not calculate the input for KEGG ID: C01653 (ind = 4157)
Could not create input for substrate ID C00464
We could not calculate the input for KEGG ID: C00464 (ind = 4249)
Could not create input for substrate ID C00996
We could not calculate the input for KEGG ID: C00996 (ind = 4285)
Could not create input for substrate ID C01322
We could not calculate the input for KEGG ID: C01322 (ind = 4288)
Could not create input for substrate ID C00462
We could not calculate the input for KEGG ID: C00462 (ind = 4289)
Could not create input for substrate ID C17023
We could not calculate the input for KEGG ID: C17

Get list with all MetaNetX IDs for which we could calculate the input for the GNN:

In [13]:
metanetX_IDs_with_graphs = os.listdir(join(datasets_dir, "BiGG_data", "input_data"))
metanetX_IDs_with_graphs = [ID.split("_")[0] for ID in metanetX_IDs_with_graphs]
metanetX_IDs_with_graphs = list(set(metanetX_IDs_with_graphs))

df_KM_all_models["substrate_available"] = False
for ind in df_KM_all_models.index:
    if df_KM_all_models["metanetx ID"][ind] in metanetX_IDs_with_graphs:
        df_KM_all_models["substrate_available"][ind] = True
        
        
df_subs["substrate_available"] = False
for ind in df_subs.index:
    if df_subs["metanetx ID"][ind] in metanetX_IDs_with_graphs:
        df_subs["substrate_available"][ind] = True
        
        
df_subs.to_pickle(join(datasets_dir, "BiGG_data", "df_subs.pkl"))
df_KM_all_models.to_pickle(join(datasets_dir, "BiGG_data", "df_KM_all_models.pkl"))

np.sum(df_subs["substrate_available"]), np.sum(df_KM_all_models["substrate_available"])/len(df_KM_all_models)

(3396, 0.8711253252485291)

#### Calculate GNN representations for all MetaNetX IDs with graphs with the Notebook "02 Calculating GNN Fingerprints for all substrates.ipynb" and load the saved results:

In [14]:
df_subs_avail = pd.read_pickle(join(datasets_dir, "BiGG_data", "df_subs_avail.pkl"))

metanetX_IDs_with_gnn_fp = list(set(df_subs_avail["metanetx ID"]))

df_KM_all_models["GNN FP"] = ""
for ind in df_KM_all_models.index:
    meta_id = df_KM_all_models["metanetx ID"][ind]
    if meta_id in metanetX_IDs_with_gnn_fp:
        df_KM_all_models["GNN FP"][ind] = list(df_subs_avail["GNN FP"].loc[df_subs_avail["metanetx ID"] == meta_id])[0]
        
df_KM_all_models.to_pickle(join(datasets_dir, "BiGG_data", "df_KM_all_models.pkl"))
df_KM_all_models

,BiGG reaction ID,reaction name,gene_reaction_rule,substrate,Uniprot ID,KEGG ID,bigg.metabolite,metanetx ID,substrate name,SMILES,Sequence,model_ID,substrate_available,GNN FP
0,PFK,Phosphofructokinase,b3916,atp_c,P0A796,C00002,atp,MNXM3,ATP C10H12N5O13P3,NaN,MIKKIGVLTSGGDAPGMNAAIRGVVRSALTEGLEVMGIYDGYLGLY...,e_coli_core,True,"[3.5004892, 0.0, 0.42342216, 0.023474924, 0.26..."
1,PFK,Phosphofructokinase,b3916,f6p_c,P0A796,C05345,f6p,MNXM89621,D-Fructose 6-phosphate,NaN,MIKKIGVLTSGGDAPGMNAAIRGVVRSALTEGLEVMGIYDGYLGLY...,e_coli_core,True,"[1.8386872, 0.0, 0.0, 0.007824975, 0.0, 1.4801..."
2,PFK,Phosphofructokinase,b1723,atp_c,P06999,C00002,atp,MNXM3,ATP C10H12N5O13P3,NaN,MVRIYTLTLAPSLDSATITPQIYPEGKLRCTAPVFEPGGGGINVAR...,e_coli_core,True,"[3.5004892, 0.0, 0.42342216, 0.023474924, 0.26..."
3,PFK,Phosphofructokinase,b1723,f6p_c,P06999,C05345,f6p,MNXM89621,D-Fructose 6-phosphate,NaN,MVRIYTLTLAPSLDSATITPQIYPEGKLRCTAPVFEPGGGGINVAR...,e_coli_core,True,"[1.8386872, 0.0, 0.0, 0.007824975, 0.0, 1.4801..."
4,PFL,Pyruvate formate lyase,((b0902 and b0903) and b2579),coa_c,NaN,C00010,coa,MNXM12,Coenzyme A,NaN,NaN,e_coli_core,True,"[4.6419253, 0.0, 0.42342216, 0.039124876, 0.27..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216368,r_4693,alcohol acetyltransferase (propanol),YGR177C,s_4233[c],P53296,C05979,NaN,NaN,propanol [cytoplasm],NaN,MEDIEGYEPHITQELIDRGHARRMGHLENYFAVLSRQKMYSNFTVY...,yeast8,False,
216369,r_4693,alcohol acetyltransferase (propanol),YOR377W,s_0373[c],P40353,C00024,a,M,acetyl-CoA [cytoplasm],NaN,MNEIDEKNQAPVQQECLKEMIQNGHARRMGSVEDLYVALNRQNLYR...,yeast8,True,"[0.08515112, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
216370,r_4693,alcohol acetyltransferase (propanol),YOR377W,s_4233[c],P40353,C05979,NaN,NaN,propanol [cytoplasm],NaN,MNEIDEKNQAPVQQECLKEMIQNGHARRMGSVEDLYVALNRQNLYR...,yeast8,False,
216371,r_4693,alcohol acetyltransferase (propanol),YGR015C,s_0373[c],P53208,C00024,a,M,acetyl-CoA [cytoplasm],NaN,MSRLAHNKALPYKIIVDLSFHRTRLPSDVSSLIKFEQRPAIINIHG...,yeast8,True,"[0.08515112, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


### Calculate UniRep vectors for all sequences in the data set:

In [15]:
df_subs = pd.read_pickle(join(datasets_dir, "BiGG_data", "df_subs.pkl"))
df_KM_all_models = pd.read_pickle(join(datasets_dir, "BiGG_data", "df_KM_all_models.pkl"))

df_enzymes = pd.DataFrame({"Uniprot ID" : df_KM_all_models["Uniprot ID"], "Sequence" : df_KM_all_models["Sequence"]})
df_enzymes = df_enzymes.loc[~pd.isnull(df_enzymes["Uniprot ID"])]
df_enzymes = df_enzymes.drop_duplicates().reset_index(drop = True)
df_enzymes

,Uniprot ID,Sequence
0,P0A796,MIKKIGVLTSGGDAPGMNAAIRGVVRSALTEGLEVMGIYDGYLGLY...
1,P06999,MVRIYTLTLAPSLDSATITPQIYPEGKLRCTAPVFEPGGGGINVAR...
2,P0A6T1,MKNINPTQTAAWQALQKHFDEMKDVTIADLFAKDGDRFSKFSATFD...
3,P0A799,MSVIKMTDLDLAGKRVFIRADLNVPVKDGKVTSDARIRASLPTIEL...
4,P52697,MKQTVYIASPESQQIHVWNLNHEGALTLTQVVDVPGQVQPMVVSPD...
...,...,...
24410,Q04119,MVVVGKSEVRNVSMSRPKKKSLIAILSTCVLFFLVFIIGAKFQYVS...
24411,P39979,MSNEEPEKMVNDRIVVKAIEPKDEEAWNKLWKEYQGFQKTVMPPEV...
24412,P38295,MSEVSKWPAINPFHWGYNGTVSHIVGENGSIKLHLKDNKEQVDFDE...
24413,Q02891,MFRSGYYPTVTPSHWGYNGTVKHVLGEKGTKSLAFRDSKRQIPLHE...


In [16]:
ofile = open(join(datasets_dir, "BiGG_data", "fasta_for_UniRep_UNIPROT_kcat.fasta"), "w")
for ind in df_enzymes.index:
    seq = df_enzymes["Sequence"][ind]
    if not pd.isnull(seq):
        seq_end = seq.find("#")
        ofile.write(">" + str(ind) + "\n" +seq[:seq_end] + "\n")
ofile.close()

In [17]:
def add_Unirep_vector(df, Unirep_df):
    X_Unirep = Unirep_df.values
    df["Unirep"] = ""
    for i in range(len(X_Unirep)):
        ind = X_Unirep[i,0]
        df["Unirep"][ind] = X_Unirep[i,1:]
    return(df)

In [18]:
Unirep_df = pd.read_csv(join(datasets_dir, "BiGG_data", "unirep_for_all_BiGG_models.tsv"), sep = "\t")
df_enzymes = add_Unirep_vector(df_enzymes, Unirep_df)
df_enzymes.head()

,Uniprot ID,Sequence,Unirep
0,P0A796,MIKKIGVLTSGGDAPGMNAAIRGVVRSALTEGLEVMGIYDGYLGLY...,"[0.0060782963, 0.24019428, 0.16744998, -0.0425..."
1,P06999,MVRIYTLTLAPSLDSATITPQIYPEGKLRCTAPVFEPGGGGINVAR...,"[0.011336103, 0.14450617, 0.121701956, -0.0309..."
2,P0A6T1,MKNINPTQTAAWQALQKHFDEMKDVTIADLFAKDGDRFSKFSATFD...,"[0.0027934216999999997, 0.08215925, 0.12447391..."
3,P0A799,MSVIKMTDLDLAGKRVFIRADLNVPVKDGKVTSDARIRASLPTIEL...,"[0.0043981396999999995, 0.21119659, 0.19810545..."
4,P52697,MKQTVYIASPESQQIHVWNLNHEGALTLTQVVDVPGQVQPMVVSPD...,"[0.0063358434, 0.0017107445, 0.049509652, -0.0..."


Map Unirep vectors to df_KM_all_models and add the column "UniRep available":

In [19]:
UIDs_with_UniRep = list(df_enzymes["Uniprot ID"])

df_KM_all_models["UniRep"] = ""
df_KM_all_models["enzyme_available"] = False
for ind in df_KM_all_models.index:
    UID = df_KM_all_models["Uniprot ID"][ind]
    if not pd.isnull(UID):
        if UID in UIDs_with_UniRep:
            df_KM_all_models["UniRep"][ind] = list(df_enzymes["Unirep"].loc[df_enzymes["Uniprot ID"] == UID])[0]
            df_KM_all_models["enzyme_available"][ind] = True
        
df_KM_all_models.to_pickle(join(datasets_dir, "BiGG_data", "df_KM_all_models.pkl"))
df_KM_all_models

,BiGG reaction ID,reaction name,gene_reaction_rule,substrate,Uniprot ID,KEGG ID,bigg.metabolite,metanetx ID,substrate name,SMILES,Sequence,model_ID,substrate_available,GNN FP,UniRep,enzyme_available
0,PFK,Phosphofructokinase,b3916,atp_c,P0A796,C00002,atp,MNXM3,ATP C10H12N5O13P3,NaN,MIKKIGVLTSGGDAPGMNAAIRGVVRSALTEGLEVMGIYDGYLGLY...,e_coli_core,True,"[3.5004892, 0.0, 0.42342216, 0.023474924, 0.26...","[0.0060782963, 0.24019428, 0.16744998, -0.0425...",True
1,PFK,Phosphofructokinase,b3916,f6p_c,P0A796,C05345,f6p,MNXM89621,D-Fructose 6-phosphate,NaN,MIKKIGVLTSGGDAPGMNAAIRGVVRSALTEGLEVMGIYDGYLGLY...,e_coli_core,True,"[1.8386872, 0.0, 0.0, 0.007824975, 0.0, 1.4801...","[0.0060782963, 0.24019428, 0.16744998, -0.0425...",True
2,PFK,Phosphofructokinase,b1723,atp_c,P06999,C00002,atp,MNXM3,ATP C10H12N5O13P3,NaN,MVRIYTLTLAPSLDSATITPQIYPEGKLRCTAPVFEPGGGGINVAR...,e_coli_core,True,"[3.5004892, 0.0, 0.42342216, 0.023474924, 0.26...","[0.011336103, 0.14450617, 0.121701956, -0.0309...",True
3,PFK,Phosphofructokinase,b1723,f6p_c,P06999,C05345,f6p,MNXM89621,D-Fructose 6-phosphate,NaN,MVRIYTLTLAPSLDSATITPQIYPEGKLRCTAPVFEPGGGGINVAR...,e_coli_core,True,"[1.8386872, 0.0, 0.0, 0.007824975, 0.0, 1.4801...","[0.011336103, 0.14450617, 0.121701956, -0.0309...",True
4,PFL,Pyruvate formate lyase,((b0902 and b0903) and b2579),coa_c,NaN,C00010,coa,MNXM12,Coenzyme A,NaN,NaN,e_coli_core,True,"[4.6419253, 0.0, 0.42342216, 0.039124876, 0.27...",,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216368,r_4693,alcohol acetyltransferase (propanol),YGR177C,s_4233[c],P53296,C05979,NaN,NaN,propanol [cytoplasm],NaN,MEDIEGYEPHITQELIDRGHARRMGHLENYFAVLSRQKMYSNFTVY...,yeast8,False,,"[0.0022087584, -0.016782222, 0.06071565, -0.03...",True
216369,r_4693,alcohol acetyltransferase (propanol),YOR377W,s_0373[c],P40353,C00024,a,M,acetyl-CoA [cytoplasm],NaN,MNEIDEKNQAPVQQECLKEMIQNGHARRMGSVEDLYVALNRQNLYR...,yeast8,True,"[0.08515112, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.002904639, -0.034499153, 0.050304506, -0.01...",True
216370,r_4693,alcohol acetyltransferase (propanol),YOR377W,s_4233[c],P40353,C05979,NaN,NaN,propanol [cytoplasm],NaN,MNEIDEKNQAPVQQECLKEMIQNGHARRMGSVEDLYVALNRQNLYR...,yeast8,False,,"[0.002904639, -0.034499153, 0.050304506, -0.01...",True
216371,r_4693,alcohol acetyltransferase (propanol),YGR015C,s_0373[c],P53208,C00024,a,M,acetyl-CoA [cytoplasm],NaN,MSRLAHNKALPYKIIVDLSFHRTRLPSDVSSLIKFEQRPAIINIHG...,yeast8,True,"[0.08515112, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.009280713000000001, -0.0064755175, 0.104364...",True


## Divide DataFrame in 4 different DataFrames for the four different prediction methods

Determine with which method we predict the KM value for every single data point:
- Full model (enzyme and substrate information)
- Enzyme model (we can't create a substrate representation for the GNN, but we have the Uniprot ID for the enzyme)
- substrate model(we don't have a Uniprot ID for the enzyme, but a substrate representation)
- mean of all KM values

In [20]:
df_Km_enz_and_sub = pd.DataFrame(columns = list(df_KM_all_models.columns))
df_Km_sub = pd.DataFrame(columns = list(df_KM_all_models.columns))
df_Km_enz = pd.DataFrame(columns = list(df_KM_all_models.columns))
df_Km_mean = pd.DataFrame(columns = list(df_KM_all_models.columns))

for ind in df_KM_all_models.index:
    if df_KM_all_models["substrate_available"][ind]:
        if df_KM_all_models["enzyme_available"][ind]:
            df_Km_enz_and_sub = df_Km_enz_and_sub.append(df_KM_all_models.loc[ind], ignore_index = True)
        else:
            df_Km_sub = df_Km_sub.append(df_KM_all_models.loc[ind], ignore_index = True)
    else:
        if df_KM_all_models["enzyme_available"][ind]:
            df_Km_enz = df_Km_enz.append(df_KM_all_models.loc[ind], ignore_index = True)
        else:
            df_Km_mean = df_Km_mean.append(df_KM_all_models.loc[ind], ignore_index = True)
            
print(len(df_Km_enz_and_sub), len(df_Km_enz), len(df_Km_sub), len(df_Km_mean))
print(len(df_Km_enz_and_sub)/len(df_KM_all_models), len(df_Km_enz)/len(df_KM_all_models),
      len(df_Km_sub)/len(df_KM_all_models), len(df_Km_mean)/len(df_KM_all_models))

156135 22805 32353 5080
0.7216011239849704 0.1053966992184792 0.14952420126355875 0.02347797553299164


#### Making predictions for data points with enzyme and substrate information:

In [21]:
import xgboost as xgb

fingerprints = np.array(list(df_Km_enz_and_sub["GNN FP"]))
UniRep = np.array(list(df_Km_enz_and_sub["UniRep"]))
X = np.concatenate([fingerprints, UniRep[:,:1900]], axis = 1)

bst = pickle.load(open(join(datasets_dir, "model_weights", "xgboost_model.dat"), "rb"))
dinput= xgb.DMatrix(X)
pred = bst.predict(dinput)

df_Km_enz_and_sub["log10_KM_pred"] = pred

#### Making predictions for data points with enzyme information:

In [22]:
UniRep = np.array(list(df_Km_enz["UniRep"]))
X = UniRep[:,:1900]

bst = pickle.load(open(join(datasets_dir, "model_weights", "xgboost_model_enzyme_only.dat"), "rb"))
dinput= xgb.DMatrix(X)
pred = bst.predict(dinput)

df_Km_enz["log10_KM_pred"] = pred

#### Making predictions for data points with substrate information:

In [23]:
fingerprints = np.array(list(df_Km_sub["GNN FP"]))
X = fingerprints


bst = pickle.load(open(join(datasets_dir, "model_weights", "xgboost_model_substrate_only.dat"), "rb"))
dinput= xgb.DMatrix(X)
pred = bst.predict(dinput)

df_Km_sub["log10_KM_pred"] = pred

#### Predict with the mean for all data points where we have neithor substrate nor enzyme information:

In [24]:
df_Km_mean["log10_KM_pred"] = -0.7676345542934437 # mean over all log10 KM values in the training set

### Concat all 4 DataFrames:

In [25]:
df_Km_enz_and_sub["prediction method"] = "enzyme and substrate information"
df_Km_sub["prediction method"] = "substrate information"
df_Km_enz["prediction method"] = "enzyme information"
df_Km_mean["prediction method"] = "mean"

df_KM_all_models = pd.concat([df_Km_enz_and_sub, df_Km_sub,  df_Km_enz, df_Km_mean], ignore_index= True)
df_KM_all_models.to_pickle(join(datasets_dir, "BiGG_data", "df_KM_all_models.pkl"))

## Create excel table sheet with all models:

In [26]:
df_KM_all_models = pd.read_pickle(join(datasets_dir, "BiGG_data", "df_KM_all_models.pkl"))
model_IDs = os.listdir(join(datasets_dir, "BiGG_GSM"))

#Load BRENDA dataset with KM values
brenda_df = pd.read_pickle(join(datasets_dir, "BRENDA_data", "brenda_df_checkpoint_4.pkl"))
brenda_df["Uniprot ID"] = np.nan
for ind in brenda_df.index:
    UID_list = brenda_df["UNIPROT"][ind]
    if len(UID_list) > 0:
        brenda_df["Uniprot ID"][ind] = UID_list[0]
        
brenda_df = brenda_df.loc[~pd.isnull(brenda_df["Uniprot ID"])]
brenda_df


##Mapping KM values to Km Prediction dataset:
df_KM_all_models["log10_KM_BRENDA"] = np.nan
for ind in brenda_df.index:
    UID, CID, log10_KM = brenda_df["Uniprot ID"][ind], brenda_df["KEGG ID"][ind], brenda_df["log10_KM"][ind]
    help_df = df_KM_all_models.loc[df_KM_all_models["Uniprot ID"] == UID].loc[df_KM_all_models["KEGG ID"] == CID]
    if len(help_df)>0:
        for ind2 in help_df.index:
            df_KM_all_models["log10_KM_BRENDA"][ind2] = log10_KM

In [27]:
df_KM_all_models["KM [mM] (predicted)"] = 10**df_KM_all_models["log10_KM_pred"] 
df_KM_all_models["KM [mM] (BRENDA)"] = 10**df_KM_all_models["log10_KM_BRENDA"] 

df_KM_all_models = df_KM_all_models.drop(columns = ["KEGG ID", "SMILES", "Sequence", "substrate_available", "enzyme_available",
                           "UniRep", "GNN FP", "log10_KM_pred"])

df_KM_all_models = pd.DataFrame(data = {"BiGG reaction ID" : df_KM_all_models["BiGG reaction ID"],
                                        "reaction name" : df_KM_all_models["reaction name"],
                                        "BiGG metabolite ID" : df_KM_all_models["bigg.metabolite"],
                                        "substrate name" : df_KM_all_models["substrate name"],
                                        "MetaNetX ID" : df_KM_all_models["metanetx ID"],
                                        "UniProt ID" : df_KM_all_models["Uniprot ID"],
                                        "KM [mM] (predicted)" : df_KM_all_models["KM [mM] (predicted)"],
                                        "KM [mM] (BRENDA)" : df_KM_all_models["KM [mM] (BRENDA)"],
                                        "prediction method" : df_KM_all_models["prediction method"],
                                        "BiGG model ID" : df_KM_all_models["model_ID"]})

In [28]:
from UliPlot.XLSX import auto_adjust_xlsx_column_width

with pd.ExcelWriter(join(datasets_dir, "BiGG_data", 'output_new.xlsx')) as writer:
    for model_ID in model_IDs:
        df_KM_model = df_KM_all_models.loc[df_KM_all_models["BiGG model ID"] == model_ID]
        df_KM = df_KM_model.drop(columns = ["BiGG model ID"]).reset_index(drop = True)
        df_KM = df_KM.sort_values(by=['BiGG reaction ID']).reset_index(drop = True)
        df_KM.to_excel(writer, sheet_name = model_ID, index=False)
        auto_adjust_xlsx_column_width(df_KM, writer, sheet_name = model_ID, margin=0)
        
with pd.ExcelWriter(join(datasets_dir, "BiGG_data", 'output_new.xlsx')) as writer:
    for model_ID in model_IDs:
        df_KM_model = df_KM_all_models.loc[df_KM_all_models["BiGG model ID"] == model_ID]
        df_KM = df_KM_model.drop(columns = ["BiGG model ID"]).reset_index(drop = True)
        df_KM = df_KM.sort_values(by=['BiGG reaction ID']).reset_index(drop = True)
        df_KM.to_excel(writer, sheet_name = model_ID, index=False)
        
        sheetname = model_ID
        worksheet = writer.sheets[sheetname]  # pull worksheet object
        for idx, col in enumerate(df_KM):  # loop through all columns
            series = df_KM[col]
            max_len = max((
                series.astype(str).map(len).max(),  # len of largest item
                len(str(series.name))  # len of column name/header
                )) + 1  # adding a little extra space
            worksheet.set_column(idx, idx, min(max_len,70))
        
writer.save()